In [1]:
"""
TODOS: Make the model editable. (Add layer count adjustments and layer type adjustments for fast edits)
       Make DataSet / DataLoaders more adaptable
       Create Editable Transforms library
       

"""
import torch
import torch.nn as nn
import torch.functional as F
from torchvision import transforms, utils
import torchvision as tv
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image;

In [2]:
"""
Build a convolutional Neural Network
"""

class CNNmodel(nn.Module):
    def __init__(self):
        super(CNNmodel, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.sig = nn.Sigmoid()
        self.conv1 = nn.Conv2d(8, 16, 5)
        self.norm1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(32, 32, 7)
        self.norm2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(64, 64, 5)
        self.norm3 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2)
        self.conv4 = nn.Conv2d(128, 128, 5)
        self.norm4 = nn.BatchNorm2d(128)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(25600, 256)  # 6*6 from image dimension
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fcout = nn.Linear(64, 4)
        
        self.norm0 = nn.BatchNorm2d(8)
        self.conv0 = nn.Conv2d(3, 8, 1)
        
        self.relu = nn.ReLU()
        self.conv_drop = nn.Dropout2d(p=0.25)
        self.drop = nn.Dropout(p=0.3)
        
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        
        # Convolutions using sigmoid multipliers to mask features -
        
        # The overall structure was developed by hours (30+ of plug and play experimenting)
        
        x = self.norm0(self.relu(self.conv0(x)))
        
        x = self.pool1(torch.cat([self.norm1(self.sig(self.conv_drop(self.conv1(x))))* self.conv1(x), self.relu(self.conv1(x))], dim=1))
        
        x = self.pool1(torch.cat([self.norm2(self.sig(self.conv_drop(self.conv2(x))))* self.conv2(x), self.relu(self.conv2(x))], dim=1))
        
        x = self.pool1(torch.cat([self.norm3(self.sig(self.conv_drop(self.conv3(x))))* self.conv3(x), self.relu(self.conv3(x))], dim=1))
        
        x = self.pool1(torch.cat([self.norm4(self.sig(self.conv_drop(self.conv4(x))))* self.conv4(x), self.relu(self.conv4(x))], dim=1))
        
        # Flatten Features -
        
        x = x.view(-1, self.num_flat_features(x))
        
        # Standard Fully Connected Out -
        
        x = self.relu(self.drop(self.fc1(x)))
        x = self.relu(self.drop(self.fc2(x)))
        x = self.relu(self.drop(self.fc3(x)))
        x = self.softmax(self.fcout(x))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features   


In [3]:
"""
This section still needs completed. 

Task 1. Adapat code to fit my data set
    a. Use index to load image and y variables alongside
    b. Use Rescale transform as well as other transforms. Figure out how to apply and customize these.
    
Task 2. Create Data Loader

This resource is helpful ->


https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

"""

class ImageClassDataset(Dataset):
    """
    Base DataSet Class for generating paths from DataFrames with paths and labels
    """

    def __init__(self, data, transform=None, labels=[], path='path'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        if isinstance(data, str):
            self.data = pd.read_csv(csv_file)
        else: 
            self.data = data
        self.transform = transform
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.data['path'][idx]
        image = Image.open(img_name)
        
        labels = self.data.loc[idx, list(self.labels)]
        
        if self.transform:
            image = self.transform(image)
            
        sample = {'data': image, 'labels': torch.tensor(labels)}

        return sample

In [4]:
train = pd.read_csv('./Train_Test/Train.csv').drop('Unnamed: 0', axis=1)
test  = pd.read_csv('./Train_Test/Test.csv').drop('Unnamed: 0', axis=1)

In [5]:


model = CNNmodel().cuda()


"""
Basic Cross Entropy
"""

criterion = nn.CrossEntropyLoss()



In [6]:
train

,image_id,healthy,multiple_diseases,rust,scab,path
0,Train_0,0,0,0,1,./images/Train_0.jpg
1,Train_1,0,1,0,0,./images/Train_1.jpg
2,Train_2,1,0,0,0,./images/Train_2.jpg
3,Train_3,0,0,1,0,./images/Train_3.jpg
4,Train_4,1,0,0,0,./images/Train_4.jpg
...,...,...,...,...,...,...
1816,Train_1816,0,0,0,1,./images/Train_1816.jpg
1817,Train_1817,1,0,0,0,./images/Train_1817.jpg
1818,Train_1818,1,0,0,0,./images/Train_1818.jpg
1819,Train_1819,0,0,1,0,./images/Train_1819.jpg


In [7]:
train_set = ImageClassDataset(train[:-100], 
                              transform=transforms.Compose([transforms.Resize((256, 256)),
                                                            transforms.RandomCrop((224,224)),
                                                            transforms.RandomHorizontalFlip(),
                                                            transforms.RandomVerticalFlip(),
                                                            transforms.ColorJitter(0.05, 0.05, 0.05, 0.05),
                                                            transforms.ToTensor(),
                                                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]), 
                              labels=['healthy', 'multiple_diseases', 'rust', 'scab'])
val_set = ImageClassDataset(train[-100:].reset_index(),
                           transform= transforms.Compose([transforms.Resize((256, 256)),
                                                          transforms.CenterCrop((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
                           labels=['healthy', 'multiple_diseases', 'rust', 'scab'])

test_set = ImageClassDataset(test, 
                             transform=transforms.Compose([transforms.Resize((256, 256)),
                                                           transforms.CenterCrop((224,224)), 
                                                           transforms.ToTensor(),
                                                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))


trainloader = DataLoader(train_set, batch_size=256,
                         shuffle=True, num_workers=4)
valloader = DataLoader(val_set, batch_size=100,
                       shuffle=False, num_workers=4)

testloader = DataLoader(test_set, batch_size=128,
                        shuffle=False, num_workers=4)

In [8]:
def model_loop(num_iter):
    for epoch in range(num_iter):  # loop over the dataset multiple times

        train_loss = 0.0
        val_accuracy = 0.0
        t = tqdm(iter(trainloader), leave=False, total=len(trainloader))
        model.train()
        for i, sample in enumerate(t):
            # get the inputs; data is a list of [inputs, labels]
            images, labels = sample.values()
            img = images.cuda()
            
            labs = labels.cuda()
            # zero the parameter gradients
            opti.zero_grad()

            # forward + backward + optimize
            outputs = model(img)

            loss = criterion(outputs, labs.argmax(dim=1))

            loss.backward()
            opti.step()

            train_loss += loss

        q = tqdm(iter(valloader), leave=False, total=len(valloader))
        model.eval()
        for i, sample in enumerate(q):
            # get the inputs; data is a list of [inputs, labels]
            images, labels = sample.values()
            img = images.cuda()
            
            labs = labels.cuda()    
            preds = model(img)
            

            correct = (preds.argmax(dim=1) == labs.argmax(dim=1))
            val_accuracy = correct.sum().float() / float(len(labs))
            print(f"Val Accuracy: {val_accuracy.item()}, Epoch: {epoch+1}")

        

In [9]:
"""
This is super exciting!! PyTorch has an extremely easy implementation of AdamW! 
Which means I don't need to write custom weight decay functions!!!

Also check out lr_scheduler for SGD in the future. 

"""


opti = optim.Adam(params=model.parameters(), lr=0.001, amsgrad=True)
model_loop(40)

opti = optim.AdamW(params=model.parameters(), lr=0.0005, amsgrad=True)
model_loop(40)

opti = optim.AdamW(params=model.parameters(), lr=0.0001, amsgrad=True)
model_loop(40)

  0%|          | 0/7 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 758.00 MiB (GPU 0; 5.80 GiB total capacity; 4.27 GiB already allocated; 296.94 MiB free; 4.30 GiB reserved in total by PyTorch)

In [10]:
torch.save(model, './Torch_Model_Trained_ConvSigMultiply')

/home/houston/anaconda3/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CNNmodel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [9]:
model = torch.load('./Torch_Model_Trained_ConvSigMultiply')

In [10]:


model.eval()


total_preds = []
t = tqdm(iter(testloader), leave=False, total=len(testloader))

with torch.no_grad():
    for i, sample in enumerate(t):

        images = sample['data']
        img = images.cuda()
        preds = model(img)
    
    
        total_preds.append(preds)

In [11]:
preds = torch.cat([i for i in total_preds])

In [12]:
np_preds = preds.cpu().numpy()

In [13]:
preds_cleaned = [np.where(i>0.75, 1.0, 0.0) for i in np_preds]

In [14]:
df = pd.DataFrame(preds_cleaned)

In [15]:
df

,0,1,2,3
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0
...,...,...,...,...
1816,0.0,0.0,1.0,0.0
1817,0.0,0.0,0.0,1.0
1818,0.0,0.0,1.0,0.0
1819,1.0,0.0,0.0,0.0


In [16]:
image_id = [f'Test_{i}' for i in range(1821)]

In [17]:
df['image_id'] = image_id

In [18]:
df['healthy'] = df.loc[:, 0]

In [19]:
df

,0,1,2,3,image_id,healthy
0,0.0,0.0,1.0,0.0,Test_0,0.0
1,0.0,0.0,1.0,0.0,Test_1,0.0
2,0.0,0.0,0.0,1.0,Test_2,0.0
3,1.0,0.0,0.0,0.0,Test_3,1.0
4,0.0,0.0,1.0,0.0,Test_4,0.0
...,...,...,...,...,...,...
1816,0.0,0.0,1.0,0.0,Test_1816,0.0
1817,0.0,0.0,0.0,1.0,Test_1817,0.0
1818,0.0,0.0,1.0,0.0,Test_1818,0.0
1819,1.0,0.0,0.0,0.0,Test_1819,1.0


In [20]:
df['multiple_diseases'] = df.loc[:,1]
df['rust'] = df.loc[:,2]
df['scab'] = df.loc[:,3]

In [21]:
df

,0,1,2,3,image_id,healthy,multiple_diseases,rust,scab
0,0.0,0.0,1.0,0.0,Test_0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,Test_1,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,Test_2,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,Test_3,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,Test_4,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
1816,0.0,0.0,1.0,0.0,Test_1816,0.0,0.0,1.0,0.0
1817,0.0,0.0,0.0,1.0,Test_1817,0.0,0.0,0.0,1.0
1818,0.0,0.0,1.0,0.0,Test_1818,0.0,0.0,1.0,0.0
1819,1.0,0.0,0.0,0.0,Test_1819,1.0,0.0,0.0,0.0


In [22]:
df = df.drop([0, 1, 2, 3], axis=1)

In [23]:
df.to_csv('./submissions.csv', index=False)